# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [25]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [26]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [27]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [28]:
batch_size = 23 #experiment with the batch size
num_epochs = 50

# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
#total_frames = 30
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray
batch_label = 5

In [29]:
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,batch_label)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [30]:
# def normalized(image):
#     norm=np.zeros(image.shape,np.float32)
#     norm_rgb=np.zeros(image.shape,np.uint8)

#     b=rgb[:,:,0]
#     g=rgb[:,:,1]
#     r=rgb[:,:,2]

#     sum=b+g+r

#     norm[:,:,0]=b/sum*255.0
#     norm[:,:,1]=g/sum*255.0
#     norm[:,:,2]=r/sum*255.0

#     norm_rgb=cv2.convertScaleAbs(norm)
#     print('norm_rgb.shape',norm_rgb.shape)
#     return norm_rgb

In [31]:
def getBatchImage(source_path, batch_num, batch_size, t_seq,validation):
    batch_data,batch_labels = initialize_batch_data(batch_size)
    
    img_idx = [x for x in range(0, nb_frames)]
    for folder in range(batch_size):
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t_seq[folder + (batch_num*batch_size)].split(';')[0]))
        for idx, item in enumerate(img_idx):
            image = cv2.imread(source_path+'/'+ t_seq[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                print(iamshow)
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
#             batch_data[folder,idx] = (resized)
            batch_data[folder,idx,:,:,0] = resized[:, : , 0]/255
            batch_data[folder,idx,:,:,1] = resized[:, : , 1]/255
            batch_data[folder,idx,:,:,2] = resized[:, : , 2]/255
#             batch_data[folder,idx]=cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#             batch_data[folder,idx] = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)#normalise and feed in the image
            
        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    return batch_data,batch_labels

In [32]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        print('No of Batch : ', num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield getBatchImage(source_path, batch, batch_size, t,validation)
            

        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield getBatchImage(source_path, folder_list, batch, batch_size, t,validation)

In [33]:
# def generator1(source_path, folder_list, batch_size):
#     print( 'Source path = ', source_path, '; batch size =', batch_size)
#     img_idx = #create a list of image numbers you want to use for a particular video
#     while True:
#         t = np.random.permutation(folder_list)
#         num_batches = # calculate the number of batches
#         for batch in range(num_batches): # we iterate over the number of batches
#             batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
#             batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
#             for folder in range(batch_size): # iterate over the batch_size
#                 imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
#                 for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
#                     image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
#                     #crop the images and resize them. Note that the images are of 2 different shape 
#                     #and the conv3D will throw error if the inputs in a batch have different shapes
                    
#                     batch_data[folder,idx,:,:,0] = #normalise and feed in the image
#                     batch_data[folder,idx,:,:,1] = #normalise and feed in the image
#                     batch_data[folder,idx,:,:,2] = #normalise and feed in the image
                    
#                 batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
#             yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
#         # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [34]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
# num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [35]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization
from keras.layers import Activation, Dropout, ZeroPadding3D
from keras.layers.convolutional import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
# from keras.layers.recurrent import LSTM, GRU, Bidirectional
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN

#write your model here

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [36]:
#CNN2D + LSTM
model_c2d_lstm = Sequential()



model_c2d_lstm.add(TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2),
    activation='relu', padding='same'), input_shape=input_shape))
model_c2d_lstm.add(TimeDistributed(BatchNormalization()))
model_c2d_lstm.add(TimeDistributed(Conv2D(16, (3,3),
    kernel_initializer="he_normal", activation='relu')))
model_c2d_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_c2d_lstm.add(TimeDistributed(Conv2D(32, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(BatchNormalization()))
model_c2d_lstm.add(TimeDistributed(Conv2D(32, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_c2d_lstm.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(BatchNormalization()))
model_c2d_lstm.add(TimeDistributed(Conv2D(64, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_c2d_lstm.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(BatchNormalization()))
model_c2d_lstm.add(TimeDistributed(Conv2D(128, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_c2d_lstm.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(BatchNormalization()))
model_c2d_lstm.add(TimeDistributed(Conv2D(256, (3,3),
    padding='same', activation='relu')))
model_c2d_lstm.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_c2d_lstm.add(TimeDistributed(Flatten()))

model_c2d_lstm.add(Dropout(0.5))
model_c2d_lstm.add(LSTM(128, return_sequences=False, dropout=0.5))
model_c2d_lstm.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [37]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_10 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_11 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [38]:
# For CNN + LSTM
optimiser = Adam() #write your optimizer
model_c2d_lstm.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_c2d_lstm.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_64 (TimeDis (None, 30, 60, 60, 16)    448       
_________________________________________________________________
time_distributed_65 (TimeDis (None, 30, 60, 60, 16)    64        
_________________________________________________________________
time_distributed_66 (TimeDis (None, 30, 58, 58, 16)    2320      
_________________________________________________________________
time_distributed_67 (TimeDis (None, 30, 29, 29, 16)    0         
_________________________________________________________________
time_distributed_68 (TimeDis (None, 30, 29, 29, 32)    4640      
_________________________________________________________________
time_distributed_69 (TimeDis (None, 30, 29, 29, 32)    128       
_________________________________________________________________
time_distributed_70 (TimeDis (None, 30, 29, 29, 32)    9248      
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [39]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [59]:
# train_generator.batch_data.shape


In [41]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.20, patience=5, cooldown=1, verbose=1, mode='auto', min_delta=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [43]:
model_c2d_lstm.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 23
Source path =  ./Project_data/train ; batch size = 23
Epoch 1/50
No of Batch : No of Batch :  4
 28
29/29 [==============================] - 51s 2s/step - loss: 1.4431 - categorical_accuracy: 0.3866 - val_loss: 1.3984 - val_categorical_accuracy: 0.4700

Epoch 00001: saving model to model_init_2019-03-1709_50_31.029275/model-00001-1.44541-0.38612-1.39844-0.47000.h5
Epoch 2/50
29/29 [==============================] - 35s 1s/step - loss: 1.2112 - categorical_accuracy: 0.4519 - val_loss: 1.4506 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2019-03-1709_50_31.029275/model-00002-1.21122-0.45191-1.45056-0.40000.h5
Epoch 3/50
29/29 [==============================] - 35s 1s/step - loss: 1.1783 - categorical_accuracy: 0.4762 - val_loss: 2.1360 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to model_init_2019-03-1709_50_31.029275/model-00003-1.18088-0.47515-2.13599-0.35000.h5
Epoch 4/50
29/29 [=======


Epoch 00022: saving model to model_init_2019-03-1709_50_31.029275/model-00022-0.23084-0.93915-1.12918-0.85000.h5
Epoch 23/50
29/29 [==============================] - 33s 1s/step - loss: 0.1853 - categorical_accuracy: 0.9412 - val_loss: 1.3311 - val_categorical_accuracy: 0.6000

Epoch 00023: saving model to model_init_2019-03-1709_50_31.029275/model-00023-0.18534-0.94118-1.33112-0.60000.h5

Epoch 00023: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
Epoch 24/50
29/29 [==============================] - 35s 1s/step - loss: 0.1781 - categorical_accuracy: 0.9473 - val_loss: 0.2166 - val_categorical_accuracy: 0.9000

Epoch 00024: saving model to model_init_2019-03-1709_50_31.029275/model-00024-0.17808-0.94726-0.21662-0.90000.h5
Epoch 25/50
29/29 [==============================] - 35s 1s/step - loss: 0.1246 - categorical_accuracy: 0.9554 - val_loss: 0.0554 - val_categorical_accuracy: 1.0000

Epoch 00025: saving model to model_init_2019-03-1709_50_31.029275/model-00025-0.

29/29 [==============================] - 34s 1s/step - loss: 0.0706 - categorical_accuracy: 0.9838 - val_loss: 0.5853 - val_categorical_accuracy: 0.8000

Epoch 00044: saving model to model_init_2019-03-1709_50_31.029275/model-00044-0.07057-0.98377-0.58534-0.80000.h5
Epoch 45/50
29/29 [==============================] - 33s 1s/step - loss: 0.0631 - categorical_accuracy: 0.9919 - val_loss: 0.2150 - val_categorical_accuracy: 0.9500

Epoch 00045: saving model to model_init_2019-03-1709_50_31.029275/model-00045-0.06313-0.99189-0.21498-0.95000.h5
Epoch 46/50
29/29 [==============================] - 35s 1s/step - loss: 0.0809 - categorical_accuracy: 0.9817 - val_loss: 0.3136 - val_categorical_accuracy: 0.9500

Epoch 00046: saving model to model_init_2019-03-1709_50_31.029275/model-00046-0.08086-0.98174-0.31359-0.95000.h5
Epoch 47/50
29/29 [==============================] - 34s 1s/step - loss: 0.0645 - categorical_accuracy: 0.9919 - val_loss: 0.6348 - val_categorical_accuracy: 0.8500

Epoch 000

In [60]:
model_c2d_lstm.predict_classes(val_generator)

AttributeError: 'generator' object has no attribute 'ndim'

In [ ]:


model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 23
No of Batch :  4
Source path =  ./Project_data/train ; batch size = Epoch 1/7023

No of Batch :  28
No of Batch :  12
29/29 [==============================] - 51s 2s/step - loss: 1.9900 - categorical_accuracy: 0.2250 - val_loss: 1.5674 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2019-03-1707_42_29.939596/model-00001-1.99244-0.22474-1.56743-0.26000.h5
Epoch 2/70
29/29 [==============================] - 34s 1s/step - loss: 1.5061 - categorical_accuracy: 0.3049 - val_loss: 1.4928 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2019-03-1707_42_29.939596/model-00002-1.50606-0.30490-1.49284-0.45000.h5
Epoch 3/70
29/29 [==============================] - 34s 1s/step - loss: 1.3189 - categorical_accuracy: 0.4420 - val_loss: 0.9522 - val_categorical_accuracy: 0.6250

Epoch 00003: saving model to model_init_2019-03-1707_42_29.939596/model-00003-1.32029-0.44135-0.95223-0.62500.h5
Epoch 4

29/29 [==============================] - 33s 1s/step - loss: 0.0315 - categorical_accuracy: 0.9939 - val_loss: 0.2441 - val_categorical_accuracy: 0.9000

Epoch 00043: saving model to model_init_2019-03-1707_42_29.939596/model-00043-0.03152-0.99391-0.24410-0.90000.h5
Epoch 44/70
29/29 [==============================] - 34s 1s/step - loss: 0.0499 - categorical_accuracy: 0.9736 - val_loss: 0.3554 - val_categorical_accuracy: 0.8500

Epoch 00044: saving model to model_init_2019-03-1707_42_29.939596/model-00044-0.04991-0.97363-0.35541-0.85000.h5
Epoch 45/70
29/29 [==============================] - 34s 1s/step - loss: 0.0233 - categorical_accuracy: 0.9939 - val_loss: 0.3181 - val_categorical_accuracy: 0.9000

Epoch 00045: saving model to model_init_2019-03-1707_42_29.939596/model-00045-0.02326-0.99391-0.31810-0.90000.h5

Epoch 00045: ReduceLROnPlateau reducing learning rate to 1.6000001778593287e-06.
Epoch 46/70
29/29 [==============================] - 33s 1s/step - loss: 0.0496 - categorical